In [ ]:
import sys,os
sys.path.append("..")
import django
django.setup()
from crawlers.models import *
import pandas as pd
import datetime
from components.price_feature import GetAdj
import numpy as np
from components.backend_api import DataFilter

In [ ]:
class Backtest(GetAdj):
    def __init__(self, start_date: str, strategy, model=StockPriceTW, price_choice='close_price',
                 market=None, fee_ratio=4 / 10000, tax_ratio=3 / 1000):
        super().__init__(start_date, model, price_choice, market)
        self.strategy = strategy
        self.fee_ratio = fee_ratio
        self.tax_ratio = tax_ratio

    def get_backtest_data(self):
        backtest_data = GetAdj(self.start_date).normal_date_mode()
        backtest_data = backtest_data.reindex(self.strategy.index)
        return backtest_data

    def period_return(self, num, backtest_data, strategy):
        backtest_shift = backtest_data.shift(1)
        return_table = (backtest_data * (1 - self.fee_ratio) - (
                backtest_shift * (self.fee_ratio + self.tax_ratio))) / backtest_shift
        return_table = return_table.dropna(how='all')

        current_select = strategy.iloc[num].dropna()
        current_select = current_select[current_select]
        current_idx = current_select.index
        result = return_table[current_idx].iloc[num].mean()
        return result

    @staticmethod
    def max_drawdown(arr):
        i = np.argmax((np.maximum.accumulate(arr) - arr) / np.maximum.accumulate(arr))  # end of the period
        j = np.argmax(arr[:i])  # start of period
        data = {'max_drawdown': (1 - arr[i] / arr[j]), 'start': j, 'end': i}
        return data

    def report(self):
        backtest_data = self.get_backtest_data()
        report = {}
        benchmark = backtest_data['0050']
        benchmark_shift = benchmark.shift(1)
        report['benchmark'] = ((benchmark * (1 - self.fee_ratio) - (
                benchmark_shift * (self.fee_ratio + self.tax_ratio))) / benchmark_shift).fillna(1)
        report['date'] = backtest_data.index
        report['result'] = np.array(
            [1] + [self.period_return(i, backtest_data, self.strategy) for i in range(len(self.strategy) - 1)])
        report['cumprod_result'] = report['result'].cumprod()
        max_drawdown = self.max_drawdown(report['cumprod_result'])
        report['max_drawdown'] = max_drawdown['max_drawdown']
        max_drawdown_period = [max_drawdown['start'], max_drawdown['end']]
        report['max_drawdown_period'] = [report['date'][i] for i in max_drawdown_period]

        report['benchmark'].cumprod().plot()
        pd.Series(report['cumprod_result'], index=report['date']).plot()
        return reportesult'],index=report['date']).plot()
        return report
    
#     def plot(self):
#         df=self.report()
#         df['benchmark'].cumprod().plot()
#         pd.Series(df['cumprod_result'],index=df['date']).plot()
    

# Strategy

In [ ]:
cp_last_year_rev=DataFilter(model=MonthlyRevenueTW,fields=['stock_id','date','cp_last_year_rev'],start_date='2013-01-10').get_pivot()
cp_last_year_rev.index = pd.to_datetime(cp_last_year_rev.index).shift(5, "d")
cp_last_year_rev

In [ ]:
cond_rev=cp_last_year_rev>30
cond_rev

In [ ]:
df=DataFilter(model=StockPriceTW,fields=['stock_id','date','close_price'],start_date='2013-01-10',market=['sii', 'otc']).get_pivot()
start_date=df.index[0]
last_date = df.index[-1]
date_range = pd.date_range(start=start_date, end=last_date)
df = df.reindex(date_range).fillna(method='ffill').dropna(how='all')
df2=df.reindex(cond_rev.index)
df2

In [ ]:
cond2=(df2>0)&(df2<=100)
strategy=(cond_rev & cond2)
strategy

In [ ]:
df=Backtest(start_date='2013-01-10',strategy=strategy)
df=df.report()
df

In [ ]:
df['benchmark'].cumprod().plot()
pd.Series(df['cumprod_result'],index=df['date']).plot()

In [ ]:
rr=df['cumprod_result']
rr2=df['result']
rr2[86]

In [ ]:
import numpy as np
def maxdrawdown(arr):
    i = np.argmax((np.maximum.accumulate(arr) - arr)/np.maximum.accumulate(arr)) # end of the period
    j = np.argmax(arr[:i]) # start of period
    print(i,j)
    return (1-arr[i]/arr[j])

maxdrawdown(rr)

In [ ]:
arr=df['cumprod_result']
i = np.argmax((np.maximum.accumulate(arr) - arr)/np.maximum.accumulate(arr)) # end of the period
j = np.argmax(arr[:i]) 
# np.maximum.accumulate(arr)
j


In [ ]:
i

In [ ]:
np.maximum.accumulate(arr)

In [ ]:
(np.maximum.accumulate(arr) - arr)/np.maximum.accumulate(arr)